# Concepts

```{note}
The package is in the alpha phase of development. This notebook illustrates the concepts and a current state of development. 
Anything you see here may, and likely will, change.
```

The goal is to mimic what stars package in R can do - [https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example](https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example).

![vector data cube illustration](https://raw.githubusercontent.com/r-spatial/stars/master/images/cube3.png)

Because we need to use geometries as an index, we need to ensure that geopandas is using shapely 2.0 (beta) as a geometry engine. Shapely 1.8 geometries are not hashable, while shapely 2.0 are.

In [1]:
import geopandas
import shapely
import pandas
import xarray
import numpy

from xvec import GeometryIndex

We want to recreate [this example from stars documentation](https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example).

Load geometries using geopandas:

In [2]:
nc = geopandas.read_file("https://github.com/r-spatial/sf/raw/main/inst/gpkg/nc.gpkg")

Get the GeometryArray. Note that this also contains CRS information at this point.

In [3]:
origin = destination = nc.geometry.array

Create dimensions and dummy data.

In [4]:
mode = ["car", "bike", "foot"]
day = pandas.date_range("2015-01-01", periods=100)
hours = range(24)

In [5]:
from numpy.random import default_rng

rng = default_rng(1)
data = rng.integers(1, 100, size=(3, 100, 24, 100, 100))

Create `xarray.DataArray`.

In [6]:
arr = xarray.DataArray(data, coords=(mode, day, hours, origin, destination), dims=["mode", "day", "time", "origin", "destination"])
arr

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 100, destination: 100)>
array([[[[[47, 51, 75, ..., 15, 54, 82],
          [ 7, 68, 76, ..., 38,  3, 72],
          [50, 65, 16, ..., 53, 91, 15],
          ...,
          [30,  1, 15, ...,  6, 36, 46],
          [19, 47,  3, ..., 75, 42, 69],
          [48, 94, 34, ..., 47, 54, 86]],

         [[67,  5, 73, ..., 95, 34, 44],
          [80, 22, 89, ..., 93,  5, 41],
          [47, 70, 65, ..., 44, 82, 54],
          ...,
          [91, 33, 38, ..., 24, 40,  5],
          [78, 49,  9, ..., 45,  3,  9],
          [87, 73, 52, ..., 32, 58, 48]],

         [[22, 57, 42, ..., 24, 60,  1],
          [ 4,  3, 94, ..., 66, 40, 17],
          [87, 55, 87, ..., 41, 26, 55],
          ...,
...
          ...,
          [67, 50, 96, ..., 28, 89, 55],
          [83, 98,  3, ..., 91, 25, 14],
          [81,  7, 45, ..., 18, 78,  9]],

         [[29, 94, 53, ...,  3, 52, 47],
          [93, 43, 11, ..., 94, 71, 37],
          [80, 34, 29, ..., 12, 67, 48],
          ...,
          [31, 96, 58, ..., 71, 85, 90],
          [72, 95, 55, ..., 83, 17, 97],
          [72, 77, 15, ..., 42, 62,  9]],

         [[26, 76, 23, ..., 21, 92, 42],
          [60, 80, 37, ..., 49, 91, 88],
          [55, 64, 94, ..., 99, 91, 43],
          ...,
          [80, 85, 15, ..., 23, 42, 52],
          [66, 91, 73, ..., 10, 44, 22],
          [75, 27, 89, ..., 37, 57, 20]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...

Set two `GeometryIndex` instances for the `origin` and `destination` coordinates, respectively.

In [7]:
arr = arr.xvec.set_geom_indexes(["origin", "destination"], crs=origin.crs)

arr

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 100, destination: 100)>
array([[[[[47, 51, 75, ..., 15, 54, 82],
          [ 7, 68, 76, ..., 38,  3, 72],
          [50, 65, 16, ..., 53, 91, 15],
          ...,
          [30,  1, 15, ...,  6, 36, 46],
          [19, 47,  3, ..., 75, 42, 69],
          [48, 94, 34, ..., 47, 54, 86]],

         [[67,  5, 73, ..., 95, 34, 44],
          [80, 22, 89, ..., 93,  5, 41],
          [47, 70, 65, ..., 44, 82, 54],
          ...,
          [91, 33, 38, ..., 24, 40,  5],
          [78, 49,  9, ..., 45,  3,  9],
          [87, 73, 52, ..., 32, 58, 48]],

         [[22, 57, 42, ..., 24, 60,  1],
          [ 4,  3, 94, ..., 66, 40, 17],
          [87, 55, 87, ..., 41, 26, 55],
          ...,
...
          ...,
          [67, 50, 96, ..., 28, 89, 55],
          [83, 98,  3, ..., 91, 25, 14],
          [81,  7, 45, ..., 18, 78,  9]],

         [[29, 94, 53, ...,  3, 52, 47],
          [93, 43, 11, ..., 94, 71, 37],
          [80, 34, 29, ..., 12, 67, 48],
          ...,
          [31, 96, 58, ..., 71, 85, 90],
          [72, 95, 55, ..., 83, 17, 97],
          [72, 77, 15, ..., 42, 62,  9]],

         [[26, 76, 23, ..., 21, 92, 42],
          [60, 80, 37, ..., 49, 91, 88],
          [55, 64, 94, ..., 99, 91, 43],
          ...,
          [80, 85, 15, ..., 23, 42, 52],
          [66, 91, 73, ..., 10, 44, 22],
          [75, 27, 89, ..., 37, 57, 20]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex (crs=EPSG:4267)
    destination  GeometryIndex (crs=EPSG:4267)

In [8]:
arr.xindexes

Indexes:
    mode         PandasIndex
    day          PandasIndex
    time         PandasIndex
    origin       GeometryIndex (crs=EPSG:4267)
    destination  GeometryIndex (crs=EPSG:4267)

Access to the CRS via the index:

In [9]:
arr.xindexes["origin"].crs

<Geographic 2D CRS: EPSG:4267>
Name: NAD27
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North and central America: Antigua and Barbuda - onshore. Bahamas - onshore plus offshore over internal continental shelf only. Belize - onshore. British Virgin Islands - onshore. Canada onshore - Alberta, British Columbia, Manitoba, New Brunswick, Newfoundland and Labrador, Northwest Territories, Nova Scotia, Nunavut, Ontario, Prince Edward Island, Quebec, Saskatchewan and Yukon - plus offshore east coast. Cuba - onshore and offshore. El Salvador - onshore. Guatemala - onshore. Honduras - onshore. Panama - onshore. Puerto Rico - onshore. Mexico - onshore plus offshore east coast. Nicaragua - onshore. United States (USA) onshore and offshore - Alabama, Alaska, Arizona, Arkansas, California, Colorado, Connecticut, Delaware, Florida, Georgia, Idaho, Illinois, Indiana, Iowa, Kansas, Kentucky, Louisiana, Maine, Maryland, 

"Strict" selection of geometries.

In [10]:
arr.sel(origin=[origin[0], origin[5]])

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 2, destination: 100)>
array([[[[[47, 51, 75, ..., 15, 54, 82],
          [ 7, 61, 46, ...,  7, 65, 17]],

         [[67,  5, 73, ..., 95, 34, 44],
          [62, 80,  2, ..., 56, 38, 33]],

         [[22, 57, 42, ..., 24, 60,  1],
          [47,  3, 82, ..., 52, 56, 23]],

         ...,

         [[ 6, 83, 84, ..., 50, 77, 13],
          [66, 67, 24, ..., 60, 14, 91]],

         [[55, 85, 99, ..., 28, 13, 39],
          [76, 92, 33, ..., 18, 31, 75]],

         [[86, 47, 79, ..., 70, 99, 93],
          [86, 87, 78, ..., 66, 26, 80]]],

...

        [[[52, 34, 75, ...,  7, 84, 13],
          [34, 11, 38, ..., 70, 89, 56]],

         [[ 9, 52, 95, ..., 82, 89, 44],
          [84, 94, 76, ..., 54, 34, 49]],

         [[ 9, 64, 85, ..., 28, 44, 22],
          [15, 18, 14, ..., 47, 81, 18]],

         ...,

         [[ 9, 82, 60, ..., 64, 56, 98],
          [30, 70, 34, ..., 17, 34, 86]],

         [[29, 94, 53, ...,  3, 52, 47],
          [56, 99, 33, ..., 81, 88, 85]],

         [[26, 76, 23, ..., 21, 92, 42],
          [44, 47, 95, ..., 84,  5, 86]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex (crs=EPSG:4267)
    destination  GeometryIndex (crs=EPSG:4267)

Spatial-aware data selection using the "nearest" mode.

In [11]:
arr.sel(
    day="2015-01-01",
    time=0,
    origin=[shapely.Point(-80, 35), shapely.Point(-76, 35.6)],
    destination=[shapely.Point(-81, 36), shapely.Point(-77, 35.5)],
    method="nearest",
)

<xarray.DataArray (mode: 3, origin: 2, destination: 2)>
array([[[47, 23],
        [93, 84]],

       [[35, 46],
        [18, 68]],

       [[62, 69],
        [49, 32]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
    day          datetime64[ns] 2015-01-01
    time         int64 0
  * origin       (origin) object MULTIPOLYGON (((-79.91995239257812 34.807918...
  * destination  (destination) object MULTIPOLYGON (((-80.72651672363281 35.5...
Indexes:
    origin       GeometryIndex (crs=EPSG:4267)
    destination  GeometryIndex (crs=EPSG:4267)

Spatial-aware data selection using the "query" mode with a single geometry and a given predicate:

In [12]:
arr.sel(origin=shapely.box(-80, 35, -77, 38), method="intersects")

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 41, destination: 100)>
array([[[[[77, 84, 80, ..., 31, 71, 18],
          [30, 62, 58, ..., 26,  9, 61],
          [96, 86, 84, ..., 31, 69, 57],
          ...,
          [98, 86, 56, ..., 94, 22, 75],
          [85, 42, 89, ..., 23, 69, 10],
          [98, 90, 56, ..., 35, 26, 39]],

         [[ 7, 45, 63, ..., 82, 61, 99],
          [88, 96, 84, ...,  7, 87, 18],
          [67, 72, 77, ..., 27, 54,  5],
          ...,
          [86, 52, 62, ..., 50, 51, 28],
          [50,  8, 17, ..., 63, 93, 77],
          [95, 50, 26, ..., 56, 85, 46]],

         [[27, 67, 87, ..., 88, 63, 37],
          [27, 46, 96, ..., 26, 29, 10],
          [60, 86, 15, ...,  9, 51, 29],
          ...,
...
          ...,
          [86,  7, 13, ..., 66, 74, 38],
          [59, 33, 37, ..., 85, 92, 18],
          [90, 87, 39, ..., 15, 99, 71]],

         [[28, 62,  2, ..., 41,  2, 84],
          [70, 99,  7, ..., 55, 28, 78],
          [27, 78, 16, ..., 73, 89, 24],
          ...,
          [15, 80, 70, ..., 20, 12, 10],
          [77,  7, 58, ..., 90, 12, 78],
          [67, 89, 62, ..., 39,  8, 68]],

         [[87, 17, 38, ..., 41, 73, 36],
          [58, 93, 31, ..., 13, 78, 28],
          [30, 47, 25, ..., 92, 47, 74],
          ...,
          [12,  8, 60, ..., 86, 25, 44],
          [88, 56, 41, ..., 39, 19, 93],
          [29, 88, 17, ..., 54, 93, 57]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-78.11376953125 34.720985412...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex (crs=EPSG:4267)
    destination  GeometryIndex (crs=EPSG:4267)

"Query" mode selection with an array-like of geometries is not supported:

In [13]:
arr.sel(origin=[shapely.Point(-80, 35), shapely.Point(-76, 35.6)], method="intersects")

ValueError: selection with another method than nearest only supports a single geometry as input label.

Other operations like `isel`, `concat` and alignment work just like with any default (pandas) index:

In [ ]:
arr + arr.isel(origin=[0, 1], destination=[-2, -1])

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 2, destination: 2)>
array([[[[[108, 164],
          [  6, 144]],

         [[ 68,  88],
          [ 10,  82]],

         [[120,   2],
          [ 80,  34]],

         ...,

         [[154,  26],
          [104,  94]],

         [[ 26,  78],
          [  8, 152]],

         [[198, 186],
          [ 40, 114]]],

...

        [[[168,  26],
          [ 78,  10]],

         [[178,  88],
          [114,  20]],

         [[ 88,  44],
          [192, 184]],

         ...,

         [[112, 196],
          [168, 120]],

         [[104,  94],
          [142,  74]],

         [[184,  84],
          [182, 176]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-77.96073150634766 34.1...
Indexes:
    origin       GeometryIndex (crs=EPSG:4267)
    destination  GeometryIndex (crs=EPSG:4267)

In [ ]:
xarray.concat([arr.isel(origin=[0, 1]), arr.isel(origin=[-2, -1])], dim="origin")

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 4, destination: 100)>
array([[[[[47, 51, 75, ..., 15, 54, 82],
          [ 7, 68, 76, ..., 38,  3, 72],
          [19, 47,  3, ..., 75, 42, 69],
          [48, 94, 34, ..., 47, 54, 86]],

         [[67,  5, 73, ..., 95, 34, 44],
          [80, 22, 89, ..., 93,  5, 41],
          [78, 49,  9, ..., 45,  3,  9],
          [87, 73, 52, ..., 32, 58, 48]],

         [[22, 57, 42, ..., 24, 60,  1],
          [ 4,  3, 94, ..., 66, 40, 17],
          [81, 36, 30, ...,  5, 51,  6],
          [54, 70, 25, ..., 30, 18, 37]],

         ...,

         [[ 6, 83, 84, ..., 50, 77, 13],
          [44, 84, 52, ..., 38, 52, 47],
          [65, 26, 50, ..., 12, 21, 25],
...
          [44, 67, 26, ..., 29, 96, 92],
          [38, 63, 87, ..., 32, 89, 62],
          [ 1,  5, 28, ..., 88, 84, 58]],

         ...,

         [[ 9, 82, 60, ..., 64, 56, 98],
          [79, 62, 12, ..., 47, 84, 60],
          [83, 98,  3, ..., 91, 25, 14],
          [81,  7, 45, ..., 18, 78,  9]],

         [[29, 94, 53, ...,  3, 52, 47],
          [93, 43, 11, ..., 94, 71, 37],
          [72, 95, 55, ..., 83, 17, 97],
          [72, 77, 15, ..., 42, 62,  9]],

         [[26, 76, 23, ..., 21, 92, 42],
          [60, 80, 37, ..., 49, 91, 88],
          [66, 91, 73, ..., 10, 44, 22],
          [75, 27, 89, ..., 37, 57, 20]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex (crs=EPSG:4267)
    destination  GeometryIndex (crs=EPSG:4267)